# Simple Reinforcement Learning in Tensorflow Part 2-b: 
## Vanilla Policy Gradient Agent

THIS FILE COMES FROM THE BLOG THOUGH I HAVE TWEAKED IT A BIT SO WE CAN ACTUALLY SEE HOW THE BOT PERFORMS

This tutorial contains a simple example of how to build a policy-gradient based agent that can solve the CartPole problem. For more information, see this [Medium post](https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-2-ded33892c724#.mtwpvfi8b). This implementation is generalizable to more than two actions.

For more Reinforcement Learning algorithms, including DQN and Model-based learning in Tensorflow, see my Github repo, [DeepRL-Agents](https://github.com/awjuliani/DeepRL-Agents). 

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import gym
import matplotlib.pyplot as plt
%matplotlib inline

try:
    xrange = xrange
except:
    xrange = range

In [2]:
env = gym.make('CartPole-v0')

[2017-11-21 23:51:22,512] Making new env: CartPole-v0


### The Policy-Based Agent

In [3]:
gamma = 0.99

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [4]:
class agent():
    def __init__(self, lr, s_size, a_size, h_size):
        #These lines established the feed-forward part of the network. The agent takes a state and produces an action.
        self.state_in= tf.placeholder(shape=[None,s_size],dtype=tf.float32)
        hidden = slim.fully_connected(self.state_in, h_size, biases_initializer=None, activation_fn=tf.nn.relu)
        self.output = slim.fully_connected(hidden, a_size, activation_fn=tf.nn.softmax, biases_initializer=None)
        self.chosen_action = tf.argmax(self.output,1)

        #The next six lines establish the training proceedure. We feed the reward and chosen action into the network
        #to compute the loss, and use it to update the network.
        self.reward_holder = tf.placeholder(shape=[None],dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[None],dtype=tf.int32)
        
        self.indexes = tf.range(0, tf.shape(self.output)[0]) * tf.shape(self.output)[1] + self.action_holder
        self.responsible_outputs = tf.gather(tf.reshape(self.output, [-1]), self.indexes)

        self.loss = -tf.reduce_mean(tf.log(self.responsible_outputs)*self.reward_holder)
        
        tvars = tf.trainable_variables()
        self.gradient_holders = []
        for idx,var in enumerate(tvars):
            placeholder = tf.placeholder(tf.float32,name=str(idx)+'_holder')
            self.gradient_holders.append(placeholder)
        
        self.gradients = tf.gradients(self.loss,tvars)
        
        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders,tvars))

### Training the Agent

In [6]:
tf.reset_default_graph() #Clear the Tensorflow graph.

myAgent = agent(lr=1e-3,s_size=4,a_size=2,h_size=8) #Load the agent.

total_episodes = 5000 #Set total number of episodes to train agent on.
max_ep = 999
update_frequency = 5

init = tf.global_variables_initializer()

# Launch the tensorflow graph
sess = tf.Session()
sess.run(init)
i = 0
total_reward = []
total_lenght = []

gradBuffer = sess.run(tf.trainable_variables())
for ix,grad in enumerate(gradBuffer):
    gradBuffer[ix] = grad * 0

while i < total_episodes:
    s = env.reset()
    running_reward = 0
    ep_history = []
    for j in range(max_ep):
        #Probabilistically pick an action given our network outputs.
        a_dist = sess.run(myAgent.output,feed_dict={myAgent.state_in:[s]})
        a = np.random.choice(a_dist[0],p=a_dist[0])
        a = np.argmax(a_dist == a)

        s1,r,d,_ = env.step(a) #Get our reward for taking an action given a bandit.
        ep_history.append([s,a,r,s1])
        s = s1
        running_reward += r
        if d == True:
            print ("Got to step: {0}".format(j))
            #Update the network.
            ep_history = np.array(ep_history)
            ep_history[:,2] = discount_rewards(ep_history[:,2])
            feed_dict={myAgent.reward_holder:ep_history[:,2],
                    myAgent.action_holder:ep_history[:,1],myAgent.state_in:np.vstack(ep_history[:,0])}
            grads = sess.run(myAgent.gradients, feed_dict=feed_dict)
            for idx,grad in enumerate(grads):
                gradBuffer[idx] += grad

            if i % update_frequency == 0 and i != 0:
                feed_dict= dictionary = dict(zip(myAgent.gradient_holders, gradBuffer))
                _ = sess.run(myAgent.update_batch, feed_dict=feed_dict)
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad * 0

            total_reward.append(running_reward)
            total_lenght.append(j)
            break


        #Update our running tally of scores.
    if i % 100 == 0:
        print(np.mean(total_reward[-100:]))
    i += 1

38.0
42.27
44.41
55.89
71.03
98.97
141.35
139.81
132.76
160.29
177.3
176.53
176.98
185.05
173.73
188.36
192.1
194.75
198.66
195.82
189.73
194.26
198.03
197.94
197.37
195.2
197.9
195.49
197.19
195.75
194.12
198.28
198.19
195.34
175.77
182.28
192.22
187.41
185.01
157.82
161.13
161.32
168.05
190.21
182.47
183.69
182.13
180.23
192.42
198.76
195.43
198.6
197.53
200.0
195.78
185.62
169.86
187.31
193.43
189.79
189.79
179.16
189.67
187.32
184.1
173.18
141.15
130.98
135.77
126.89
122.12
130.21
115.52
127.39
135.03
139.02
157.28
179.23
173.26
159.75
168.06
168.44
168.1
168.17
181.35
183.99
162.73
143.38
154.05
186.54
194.98
195.71
160.08
119.82
112.26
125.29
127.66
136.16
129.77
101.15
110.98
129.24
123.66
131.24
134.69
138.32
147.3
153.58
142.92
134.74
146.74
139.32
127.09
135.95
144.49
159.91
174.07
194.28
181.26
174.98
185.23
196.2
196.21
198.38
196.47
199.35
199.44
174.65
157.0
167.51
164.57
145.84
144.62
165.2
152.12
159.1
152.65
125.77
115.2
126.57
131.21
131.68
140.07
147.24
151.81
147.28

In [14]:
# sess = tf.Session()
# sess.run(init)
# import gym
# env = gym.make('CartPole-v0')
env.reset()
observation = env.reset()
done = False
t = 0
while t<900:
    env.render()
    print(observation)
    action = np.argmax(sess.run(myAgent.output,feed_dict={myAgent.state_in:[observation]}))
    observation, reward, done, info = env.step(action)
    t += 1
    if done and False:
        print("Episode finished after {} timesteps".format(t+1))
        break
env.render(close=True)

[-0.00191552 -0.04382728  0.03044596  0.04006074]
[-0.00279207  0.15084515  0.03124717 -0.24286284]
[ 0.00022484 -0.04470887  0.02638991  0.05951018]
[-0.00066934  0.15002496  0.02758012 -0.22473112]
[ 0.00233116 -0.0454801   0.02308549  0.07652246]
[ 0.00142155  0.14930343  0.02461594 -0.20878845]
[ 0.00440762 -0.0461617   0.02044017  0.09155677]
[ 0.00348439  0.14866141  0.02227131 -0.19460785]
[ 0.00645762 -0.04677193  0.01837915  0.10501668]
[ 0.00552218  0.14808188  0.02047949 -0.1818115 ]
[ 0.00848382 -0.04732704  0.01684326  0.11726096]
[ 0.00753728  0.14754958  0.01918848 -0.17006083]
[ 0.01048827 -0.04784169  0.01578726  0.12861319]
[ 0.00953143  0.14705059  0.01835952 -0.15904754]
[ 0.01247245 -0.04832933  0.01517857  0.13937035]
[ 0.01150586  0.14657197  0.01796598 -0.1484856 ]
[ 0.0144373  -0.04880258  0.01499627  0.14981065]
[ 0.01346125  0.14610146  0.01799248 -0.13810379]
[ 0.01638328 -0.04927352  0.0152304   0.16020073]
[ 0.01539781  0.14562712  0.01843442 -0.12763874]


[ 0.28408685 -0.04985485  0.00069889  0.17300808]
[ 0.28308975  0.14525709  0.00415905 -0.11945429]
[ 0.2859949  -0.0499242   0.00176997  0.17453787]
[ 0.28499641  0.14517237  0.00526073 -0.11758616]
[ 0.28789986 -0.05002456  0.002909    0.17675183]
[ 0.28689937  0.14505564  0.00644404 -0.11501199]
[ 0.28980048  0.34008467  0.0041438  -0.40565492]
[ 0.29660217  0.1449042  -0.0039693  -0.11166846]
[ 0.29950026 -0.05016065 -0.00620267  0.17975954]
[ 0.29849705  0.14504951 -0.00260748 -0.11487364]
[ 0.30139804 -0.05003498 -0.00490495  0.17698551]
[ 0.30039734  0.14515682 -0.00136524 -0.11724072]
[ 0.30330047 -0.04994554 -0.00371005  0.17501117]
[  3.02301562e-01   1.45229308e-01  -2.09831240e-04  -1.18839843e-01]
[ 0.30520615 -0.04988964 -0.00258663  0.17377688]
[ 0.30420836  0.14526924  0.00088891 -0.11972092]
[ 0.30711374 -0.04986544 -0.00150551  0.17324231]
[ 0.30611643  0.14527803  0.00195934 -0.11991517]
[ 0.30902199 -0.04987194 -0.00043897  0.17338526]
[ 0.30802455  0.14525629  0.00

[-1.49906146 -1.31557304  0.09955377  0.01305665]
[-1.52537292 -1.51197135  0.0998149   0.33541593]
[-1.55561235 -1.31840111  0.10652322  0.07580266]
[-1.58198037 -1.51487612  0.10803927  0.4001033 ]
[-1.61227789 -1.32143931  0.11604134  0.14334165]
[-1.63870668 -1.51801543  0.11890817  0.47026243]
[-1.66906699 -1.32475617  0.12831342  0.2172972 ]
[-1.69556211 -1.13167972  0.13265937 -0.03231587]
[-1.71819571 -1.32842995  0.13201305  0.29910396]
[-1.7447643  -1.13541256  0.13799513  0.05079626]
[-1.76747256 -1.3322157   0.13901105  0.38363662]
[-1.79411687 -1.1393129   0.14668378  0.13781141]
[-1.81690313 -0.94656314  0.14944001 -0.10523783]
[-1.83583439 -1.14347552  0.14733526  0.23061077]
[-1.8587039  -0.95073245  0.15194747 -0.01221149]
[-1.87771855 -1.14766999  0.15170324  0.32429206]
[-1.90067195 -0.95499693  0.15818908  0.08303158]
[-1.91977189 -0.76245454  0.15984971 -0.15586207]
[-1.93502098 -0.95946113  0.15673247  0.18267637]
[-1.9542102  -0.76688856  0.160386   -0.05675091]


[-1.30769219  0.14884436 -0.01610899 -0.19893836]
[-1.3047153  -0.04604353 -0.02008776  0.0886197 ]
[-1.30563617  0.14936051 -0.01831537 -0.21033262]
[-1.30264896 -0.04549484 -0.02252202  0.07651696]
[-1.30355886 -0.2402868  -0.02099168  0.36200989]
[-1.3083646  -0.04487287 -0.01375148  0.06278247]
[-1.30926205 -0.23979498 -0.01249583  0.35109519]
[-1.31405795 -0.04449758 -0.00547393  0.05449827]
[-1.31494791 -0.23954061 -0.00438396  0.34544912]
[-1.31973872 -0.04435658  0.00252502  0.05138699]
[-1.32062585 -0.23951464  0.00355276  0.34486552]
[-1.32541614 -0.04444341  0.01045007  0.05330502]
[-1.32630501  0.15052715  0.01151617 -0.23606259]
[-1.32329447 -0.04475742  0.00679492  0.06023053]
[-1.32418962  0.15026645  0.00799953 -0.23030083]
[-1.32118429 -0.04496889  0.00339351  0.06489465]
[-1.32208366  0.15010424  0.00469141 -0.22671568]
[ -1.31908158e+00  -4.50844414e-02   1.57092069e-04   6.74433927e-02]
[-1.31998327  0.15003526  0.00150596 -0.22518997]
[-1.31698256 -0.04510818 -0.00

[-1.32945557 -0.04479698 -0.00304815  0.06110338]
[-1.33035151 -0.23987509 -0.00182608  0.35282304]
[-1.33514901 -0.04472722  0.00523038  0.05956485]
[-1.33604355  0.15031934  0.00642168 -0.23146327]
[-1.33303716 -0.04489378  0.00179241  0.06323833]
[-1.33393504  0.15020243  0.00305718 -0.22887854]
[-1.33093099 -0.04496307 -0.00152039  0.06476716]
[ -1.33183025e+00   1.50180642e-01  -2.25049592e-04  -2.28395075e-01]
[-1.32882664 -0.04493809 -0.00479295  0.06421685]
[-1.3297254   0.15025225 -0.00350861 -0.22997442]
[-1.32672036 -0.04481939 -0.0081081   0.06159971]
[-1.32761674 -0.23982416 -0.00687611  0.35171349]
[ -1.33241323e+00  -4.46050999e-02   1.58161654e-04   5.68702669e-02]
[ -1.33330533e+00   1.50514583e-01   1.29556699e-03  -2.35762755e-01]
[-1.33029504 -0.04462585 -0.00341969  0.05732855]
[-1.33118756 -0.23969861 -0.00227312  0.34893058]
[-1.33598153 -0.0445444   0.00470549  0.05553173]
[-1.33687242  0.15050977  0.00581613 -0.23566287]
[ -1.33386222e+00  -4.46947954e-02   1.1

[-1.34728959  0.15023192  0.00356585 -0.22952866]
[ -1.34428496e+00  -4.49408104e-02  -1.02472362e-03   6.42769241e-02]
[ -1.34518377e+00   1.50195818e-01   2.60814859e-04  -2.28729133e-01]
[-1.34217986 -0.04492986 -0.00431377  0.06403605]
[-1.34307845  0.15025367 -0.00303305 -0.23000477]
[-1.34007338 -0.0448248  -0.00763314  0.06171989]
[-1.34096988 -0.23983649 -0.00639874  0.35198477]
[ -1.34576661e+00  -4.46241291e-02   6.40951112e-04   5.72910096e-02]
[-1.34665909  0.15048863  0.00178677 -0.23518962]
[-1.34364932 -0.04465881 -0.00291702  0.05805637]
[-1.34454249 -0.23973881 -0.00175589  0.34981753]
[-1.34933727 -0.04459193  0.00524046  0.05658143]
[-1.35022911  0.15045449  0.00637209 -0.2344435 ]
[-1.34722002 -0.04475792  0.00168322  0.06024256]
[-1.34811518  0.15033985  0.00288807 -0.23190883]
[-1.34510838 -0.04482325 -0.00175011  0.06168369]
[ -1.34600484e+00   1.50323753e-01  -5.16436139e-04  -2.31550894e-01]
[-1.34299837 -0.04479081 -0.00514745  0.06096909]
[-1.34389419 -0.2398